In [1]:
%matplotlib inline

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image
from scipy.stats import linregress

In [2]:
from Solver import solve_pde

## REAL WORLD PARAMETERS:

### Current device channel dimensions:

Length: 11mm

Height: 1mm

Width: 1mm

### Initial Chemoattractant concentration:

C0 = 100 nanomolar

### Buffer Solution: Basically water

### Chemoattractant protein:

TGF-Beta

Approximate diffusion constant from particle size 

#### 

r = (1.9)*(1/1000000000) #m 
Radius of spherical particle --> https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3055910/ AND https://www.abcam.com/recombinant-human-tgf-beta-1-protein-active-ab50036.html#:~:text=TGF%2Dbeta1%20is%20a%2025.0,peptide%20and%20latency%2Dassociated%20peptide.

D = (k*T)(6pi*mu*r)

Where k is boltzmann constant (1.380649 * 10^-23 J/K), T is temperature (K), mu is dynamic visc of water (.0006922 Pa * s),
and r is particle radius


## Simulation data for existing product IBIDI Mu-Slide Chemotaxis Chamber

### IBIDI device channel dimensions:

Length: 1mm

Width: 2mm

Height: .07mm

### IBIDI Device does not have any nanofibers, disregard adsoprtion

Ibidi device time to linearize = 26 hours

All other parameters (chemoattractant, buffer, C0) should be same

In [3]:
def time_to_linearize(C_profile, dist_array, desired_linearity):
    '''
    C_profile is list of concentration profile arrays at each time
    
    C_profile = [C_timestamp(t=0), C_timestamp(t=1),..]
    C_timestamp = [C(x=0),C(x=1),....]
    
    dist_array = list of distances. Single 1-D array
    '''
    
    for times, C_timestamp in enumerate(C_profile):
        
        try:
            regress_results = linregress(C_timestamp, dist_array)
            linearity = 1 - regress_results.stderr
        except:
            linearity = 0
        
        if linearity >= desired_linearity:
            return times
        else:
            continue
    
    return False
    '''
    Returns time as an integer relative to overall time function/segmentation.
    
    If C_profile is created with 60 time intervals, and this function returns '34':
    That means linearization was achieved at time 34/60 * overall time. 
    
    returns False if desired linearity never achieved
    '''

In [4]:
def linearization(linearity = .99, alpha = 5, beta = 1, n = 3):
    ## Inputs
    '''
    # Dimensionless Parameters
    alpha = 5.  # k*L^2/D
    beta = 1.   # f*(S0/C0)^(1/n)
    n = 3.      # n
    '''

    # Solver parameters
    seg = 101   # Number of length segments
    tf = 2.0  # Final time

    # Boundary conditions
    theta_BC = np.array((1., 0.))

    # Initial conditions
    theta_IC = np.zeros(2 * seg)
    
    # Solve PDEs
    sol = solve_pde(tf, theta_IC, theta_BC, alpha)

    # Unpack solution
    t = sol.t
    theta = sol.y[:int(len(sol.y) / 2)]
    theta_BL = sol.y[int(len(sol.y) / 2):]
    
    t_Bulk = time_to_linearize(theta.T, np.linspace(0, 1, seg), linearity)
    t_BL = time_to_linearize(theta_BL.T, np.linspace(0, 1, seg), linearity)
    
    return [t_Bulk, t_BL]

In [5]:
linearization()

[219, 341]

# Morris Sensitivity Analysis

In [6]:
import SALib

In [7]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp

In [8]:
morris_problem = {
    'num_vars': 3,
    'names': ['alpha','beta','n'],
    'bounds': [[15, 90],
               [8000,20000],
               [.1, 20]], 
    'groups': None
}

In [9]:
num_levels = 4
trajectories = 1000
sample = ms.sample(morris_problem, trajectories, num_levels = num_levels)
sample.shape
output = sample.T

In [10]:
Si = ma.analyze(morris_problem,
               sample,
               output,
               print_to_console=False,
               num_levels=num_levels)
print("{:20s} {:>7s} {:>7s} {:>7s}".format("Name", "mu", "mu_star", "sigma"))
for name, s1, st, mean in zip(morris_problem['names'], Si['mu'], Si['mu_star'], Si['sigma']):
    print("{:20s} {:=7.2f} {:=7.2f} {:=7.2f}".format(name, s1, st, mean))
    
fig, (ax1, ax2) = plt.subplots(1,2)
mp.horizontal_bar_plot(ax1, Si) #  param_dict={}
mp.covariance_plot(ax2, Si, {})

ValueError: cannot reshape array of size 12000 into shape (1000,4)